In [154]:
import numpy as np
import cv2
import scipy.io
import h5py 
import os
from sklearn.utils import shuffle

In [55]:
def getpatch(x,i,j):
    return x[i-10:i+11,j-10:j+11]

In [95]:
im = cv2.imread('C:\\Users\\ADMIN\\Desktop\\AmishMegh\\EdgeDetection\\BSR\\BSDS500\\data\\images\\train\\12003.jpg')

newim = cv2.copyMakeBorder(im,10,10,10,10,cv2.BORDER_CONSTANT,value=[255,255,255])


total = 1012224
ones = 506113
zeros = total - ones

(array([ 33, 105, 165, 180, 228, 232, 233, 262, 313, 319, 320, 327, 328],
      dtype=int64), array([108, 420, 436,  46, 149,  25,  25, 314, 310, 164, 329, 263, 263],
      dtype=int64), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64))


In [100]:
mat = scipy.io.loadmat('C:\\Users\\ADMIN\\Desktop\\AmishMegh\\EdgeDetection\\BSR\\BSDS500\\data\\groundTruth\\train\\12003.mat')
mat = mat['groundTruth'][0][0][0][0][1]
print(np.unique(mat,return_counts=True))

a,b = np.where(mat == 1)
print(a,b)

(array([0, 1], dtype=uint8), array([152562,   1839], dtype=int64))
[  0   0   0 ... 318 319 320] [ 70 123 124 ... 414 415 416]


In [159]:
data = []
labels = []

path1 = 'C:\\Users\\ADMIN\\Desktop\\AmishMegh\\EdgeDetection\\BSR\\BSDS500\\data\\groundTruth\\train'
path2 = 'C:\\Users\\ADMIN\\Desktop\\AmishMegh\\EdgeDetection\\BSR\\BSDS500\\data\\images\\train'

In [160]:
for aFile in os.listdir(path1):
    gTruth = scipy.io.loadmat(path1+"\\"+aFile)
    gTruth = gTruth['groundTruth'][0][0][0][0][1]
    
    img = cv2.imread(path2+"\\"+aFile[0:len(aFile)-4]+".jpg")    
    img = cv2.copyMakeBorder(img,10,10,10,10,cv2.BORDER_CONSTANT,value=[255,255,255])
    
    if img.shape == (501,341,3):
        img = cv2.resize(img,(501,341))
        gTruth = gTruth.reshape(321,481)
    i_s,j_s = np.where(gTruth == 1)
    
    for i in range(len(i_s)):
        aPatch = getpatch(img,i_s[i]+10,j_s[i]+10)
        data.append(aPatch)
        labels.append(1)
    


In [161]:
#200*2531
x = []
for aFile in os.listdir(path1):
    gTruth = scipy.io.loadmat(path1+"\\"+aFile)
    gTruth = gTruth['groundTruth'][0][0][0][0][1]
    
    img = cv2.imread(path2+"\\"+aFile[0:len(aFile)-4]+".jpg")    
    img = cv2.copyMakeBorder(img,10,10,10,10,cv2.BORDER_CONSTANT,value=[255,255,255])
    
    if img.shape == (501,341,3):
        img = cv2.resize(img,(501,341))
        gTruth = gTruth.reshape(321,481)
    i_s,j_s = np.where(gTruth == 0)
    
    i_s,j_s = shuffle(i_s,j_s)
    
    for i in range(2531):
        aPatch = getpatch(img,i_s[i]+10,j_s[i]+10)
        data.append(aPatch)
        labels.append(0)

In [162]:
labels = np.asarray(labels)
data = np.asarray(data)
print(labels.shape,data.shape)

(1012313,) (1012313, 21, 21, 3)


In [164]:

with h5py.File('inputs.h5','w') as f:
    f['data'] = data[0:total,:,:,:]
    f['label'] = labels[0:total]